# mastery data

In [3]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [4]:
path = '../data/mastery_data.json'

In [5]:
import json

dfs = []

with open(path) as f:
    data = json.load(f)

for d in data:
    dfs.append(pd.json_normalize(d))

mastery_data = pd.concat(dfs, ignore_index=True)

In [6]:
mastery_data = mastery_data.loc[:, ~mastery_data.columns.str.startswith('nextSeasonMilestone')] # remoção de todos os desafios
mastery_data = mastery_data.drop(columns=['markRequiredForNextLevel', 'tokensEarned', 'championSeasonMilestone', 'milestoneGrades'])\



mastery_data['lastPlayTime'] = pd.to_datetime(mastery_data['lastPlayTime'], unit='ms',  utc=True).dt.tz_convert('America/Sao_Paulo')

mastery_data.info()


<class 'pandas.DataFrame'>
RangeIndex: 1077 entries, 0 to 1076
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype                            
---  ------                        --------------  -----                            
 0   puuid                         1077 non-null   str                              
 1   championId                    1077 non-null   int64                            
 2   championLevel                 1077 non-null   int64                            
 3   championPoints                1077 non-null   int64                            
 4   lastPlayTime                  1077 non-null   datetime64[ms, America/Sao_Paulo]
 5   championPointsSinceLastLevel  1077 non-null   int64                            
 6   championPointsUntilNextLevel  1077 non-null   int64                            
dtypes: datetime64[ms, America/Sao_Paulo](1), int64(5), str(1)
memory usage: 59.0 KB


In [65]:
mastery_data.columns

Index(['puuid', 'championId', 'championLevel', 'championPoints',
       'lastPlayTime', 'championPointsSinceLastLevel',
       'championPointsUntilNextLevel'],
      dtype='str')

# match data -> players / metadatas of matches

In [2]:
path_matches = '../data/latest_matches_data.json'

In [4]:
import json
import pandas as pd

matches = []

with open(path_matches) as f:
    data = json.load(f)

for d in data:
    matches.append(pd.json_normalize(d))

matches_data_full = pd.concat(matches, ignore_index=True)


In [ ]:
matches_data = pd.DataFrame()

matches_data['match_id'] = matches_data_full['metadata.matchId']
matches_data['creation_time'] = matches_data_full['info.gameCreation']
matches_data['duration'] = matches_data_full['info.gameDuration']
matches_data['end_time'] = matches_data_full['info.gameEndTimestamp']

matches_data['creation_time'] = pd.to_datetime(matches_data['creation_time'], unit='ms',  utc=True).dt.tz_convert('America/Sao_Paulo')
matches_data['end_time'] = pd.to_datetime(matches_data['end_time'], unit='ms',  utc=True).dt.tz_convert('America/Sao_Paulo')
matches_data['duration'] = (
    matches_data["duration"] / 60
).round(2)

matches_data

,match_id,creation_time,duration,end_time
0,BR1_3207143900,2026-02-16 23:19:51.054000-03:00,39.03,2026-02-17 00:00:03.438000-03:00
1,BR1_3191052191,2026-01-13 00:44:50.448000-03:00,27.25,2026-01-13 01:13:01.139000-03:00
2,BR1_3207120909,2026-02-16 22:38:16.295000-03:00,34.83,2026-02-16 23:13:48.562000-03:00
3,BR1_3207169625,2026-02-17 00:05:46.800000-03:00,23.33,2026-02-17 00:29:47.881000-03:00
4,BR1_3207143900,2026-02-16 23:19:51.054000-03:00,39.03,2026-02-17 00:00:03.438000-03:00
5,BR1_3207143900,2026-02-16 23:19:51.054000-03:00,39.03,2026-02-17 00:00:03.438000-03:00
6,BR1_3207194868,2026-02-17 01:04:35.289000-03:00,31.98,2026-02-17 01:37:09.498000-03:00


In [6]:
matches_data.info()

<class 'pandas.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype                            
---  ------         --------------  -----                            
 0   match_id       7 non-null      str                              
 1   creation_time  7 non-null      datetime64[ms, America/Sao_Paulo]
 2   duration       7 non-null      int64                            
 3   end_time       7 non-null      datetime64[ms, America/Sao_Paulo]
dtypes: datetime64[ms, America/Sao_Paulo](2), int64(1), str(1)
memory usage: 356.0 bytes


In [22]:
teams_exploded = matches_data_full.explode('info.teams')
teams = pd.json_normalize(teams_exploded['info.teams'])
teams['match_id'] = matches_data_full['metadata.matchId']

teams

cols = [
        'bans',
        'objectives.atakhan.first',
        'objectives.atakhan.kills',
        'objectives.baron.first',
        'objectives.dragon.first',
        'objectives.horde.first',
        'objectives.inhibitor.first',
        'objectives.riftHerald.first',
        'feats.EPIC_MONSTER_KILL.featState',
        'feats.FIRST_BLOOD.featState',
        'feats.FIRST_TURRET.featState'
    ]  

teams = teams.drop(columns=cols)

teams.info()


<class 'pandas.DataFrame'>
Index: 14 entries, 0 to 6
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   teamId                       14 non-null     int64
 1   win                          14 non-null     bool 
 2   objectives.baron.kills       14 non-null     int64
 3   objectives.champion.first    14 non-null     bool 
 4   objectives.champion.kills    14 non-null     int64
 5   objectives.dragon.kills      14 non-null     int64
 6   objectives.horde.kills       14 non-null     int64
 7   objectives.inhibitor.kills   14 non-null     int64
 8   objectives.riftHerald.kills  14 non-null     int64
 9   objectives.tower.first       14 non-null     bool 
 10  objectives.tower.kills       14 non-null     int64
 11  match_id                     14 non-null     str  
dtypes: bool(3), int64(8), str(1)
memory usage: 1.1 KB


# Players

In [11]:
exploded = matches_data_full.explode('info.participants')
players = pd.json_normalize(exploded['info.participants'])


In [13]:
# remoção de colunas não importantes
cols = ['PlayerScore0'
                            ,'PlayerScore1',
                                'PlayerScore2',
                                'PlayerScore3',
                                'PlayerScore4',
                                'PlayerScore5',
                                'PlayerScore6',
                                'PlayerScore7',
                                'PlayerScore8',
                                'PlayerScore9',
                                'PlayerScore10',
                                'PlayerScore11',
                                'championId',
                                'championTransform',
                                'eligibleForProgression',
                                'gameEndedInEarlySurrender',
                                'gameEndedInSurrender',
                                'item0',
                                'item1',
                                'item2',
                                'item3',
                                'item4',
                                'item5',
                                'item6',
                                'lane',
                                'nexusKills',
                                'nexusLost',
                                'nexusTakedowns',
                                'participantId',
                                'placement',
                                'playerAugment1',
                                'playerAugment2',
                                'playerAugment3',
                                'playerAugment4',
                                'playerAugment5',
                                'playerAugment6',
                                'playerSubteamId',
                                'profileIcon',
                                'role',
                                'spell1Casts',
                                'spell2Casts',
                                'spell3Casts',
                                'spell4Casts',
                                'subteamPlacement',
                                'summoner1Casts',
                                'summoner1Id',
                                'summoner2Casts',
                                'summoner2Id',
                                'summonerId',
                                'summonerLevel',
                                'summonerName',
                                'teamEarlySurrendered',
                                'teamPosition',
                                'timeCCingOthers',
                                'unrealKills',
                                ]

# remoção de colunas não importantes
players = players.loc[:, ~players.columns.str.startswith('challenges.')]
players = players.loc[:, ~players.columns.str.startswith('missions')]
players = players.loc[:, ~players.columns.str.startswith('perks.')]
players = players.drop(columns=cols)

# adição da coluna match_id que vai referenciar a partida
players['match_id'] = matches_data['match_id']


KeyError: "['PlayerScore0', 'PlayerScore1', 'PlayerScore2', 'PlayerScore3', 'PlayerScore4', 'PlayerScore5', 'PlayerScore6', 'PlayerScore7', 'PlayerScore8', 'PlayerScore9', 'PlayerScore10', 'PlayerScore11', 'championId', 'championTransform', 'eligibleForProgression', 'gameEndedInEarlySurrender', 'gameEndedInSurrender', 'item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6', 'lane', 'nexusKills', 'nexusLost', 'nexusTakedowns', 'participantId', 'placement', 'playerAugment1', 'playerAugment2', 'playerAugment3', 'playerAugment4', 'playerAugment5', 'playerAugment6', 'playerSubteamId', 'profileIcon', 'role', 'spell1Casts', 'spell2Casts', 'spell3Casts', 'spell4Casts', 'subteamPlacement', 'summoner1Casts', 'summoner1Id', 'summoner2Casts', 'summoner2Id', 'summonerId', 'summonerLevel', 'summonerName', 'teamEarlySurrendered', 'teamPosition', 'timeCCingOthers', 'unrealKills'] not found in axis"

In [21]:
players.dtypes.to_frame("dtype")

,dtype
allInPings,int64
assistMePings,int64
assists,int64
baronKills,int64
basicPings,int64
champExperience,int64
champLevel,int64
championName,str
commandPings,int64
consumablesPurchased,int64
